In [9]:
# Se importan todas las librerias que usaremos
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import sqlalchemy
import pymysql
from keys import *

In [10]:
# Configuración para Barcelona (LEBL)
icao_code = "LEBL"

In [11]:
# Fechas - usar fechas futuras (ajusta según necesites)
start_date = datetime.now().date() + timedelta(days=1)  # Mañana
end_date = start_date

# Formatear fechas para la API
start_time = f"{start_date}T08:00"
end_time = f"{end_date}T20:00"

In [12]:
# Este es el endpoint que usaremos para obtener los vuelos
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao_code}/{start_time}/{end_time}"

# Estos son los parametros que usaremos en la consulta
querystring = {
    "withLeg":"true",
    "direction":"Arrival",
    "withCancelled":"false",
    "withCodeshared":"true",
    "withCargo":"true",
    "withPrivate":"false",
    "withLocation":"false"
}

# Estos son los headers que usaremos en la consulta
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
}

# Se llama a la API o consulta
response = requests.request("GET", url, headers=headers, params=querystring)

In [13]:
# Verificar respuesta
if response.status_code == 200:
    flight_data = response.json()
    print("Datos obtenidos correctamente de la API")
else:
    print(f"Error en la API: {response.status_code}")
    print(response.text)
    flight_data = None

Datos obtenidos correctamente de la API


In [17]:
# Procesar los datos si la respuesta fue exitosa
if flight_data and 'arrivals' in flight_data:
    # Lista para almacenar los vuelos procesados
    flights_list = []
    
    print("Procesando datos de vuelos...")
    
    for flight in flight_data['arrivals']:
        # DEBUG: Ver la estructura de un vuelo
        if len(flights_list) == 0:  # Solo para el primer vuelo
            print("Estructura del primer vuelo:")
            print(f"  - scheduledTime: {flight.get('arrival', {}).get('scheduledTime', {})}")
            print(f"  - Tipo: {type(flight.get('arrival', {}).get('scheduledTime', {}))}")
        
        # EXTRAER CORRECTAMENTE la hora UTC
        scheduled_time = flight.get('arrival', {}).get('scheduledTime', {})
        
        # Si scheduled_time es un diccionario, tomar la clave 'utc'
        if isinstance(scheduled_time, dict):
            scheduled_time_utc = scheduled_time.get('utc')
        else:
            scheduled_time_utc = scheduled_time
        
        # Extraer otros datos
        flight_info = {
            'scheduled_arrival_time': scheduled_time_utc,
            'flight_number': flight.get('number', 'N/A'),
            'from_airport': flight.get('departure', {}).get('airport', {}).get('name', 'Unknown'),
            'airline': flight.get('airline', {}).get('name', 'Unknown'),
            'aircraft': flight.get('aircraft', {}).get('model', 'Unknown'),
            'icao_code': icao_code
        }
        flights_list.append(flight_info)
    
    # Crear DataFrame
    flights_df = pd.DataFrame(flights_list)
    
    print(f"DataFrame creado con {len(flights_df)} filas")
    
    # Verificar valores antes de convertir
    print("Primeros valores de scheduled_arrival_time:")
    print(flights_df['scheduled_arrival_time'].head())
    print(f"Tipos: {flights_df['scheduled_arrival_time'].apply(type).unique()}")
    
    # Limpiar datos nulos
    initial_count = len(flights_df)
    flights_df = flights_df.dropna(subset=['scheduled_arrival_time'])
    final_count = len(flights_df)
    
    if initial_count != final_count:
        print(f"Se eliminaron {initial_count - final_count} filas con scheduled_arrival_time nulo")
    
    # Convertir a datetime - FORMA SEGURA
    if not flights_df.empty:
        try:
            flights_df['scheduled_arrival_time'] = pd.to_datetime(
                flights_df['scheduled_arrival_time'], 
                errors='coerce'  # Convierte errores a NaT
            )
            
            # Verificar conversión
            successful_conversions = flights_df['scheduled_arrival_time'].notna().sum()
            print(f"{successful_conversions}/{len(flights_df)} fechas convertidas exitosamente")
            
            # Eliminar filas donde la conversión falló
            flights_df = flights_df.dropna(subset=['scheduled_arrival_time'])
            
        except Exception as e:
            print(f"Error en conversión de fechas: {e}")
    
    print("Datos finales:")
    print(flights_df.head())
    
else:
    print("No se encontraron datos de llegadas o hubo un error")
    flights_df = pd.DataFrame()


Procesando datos de vuelos...
Estructura del primer vuelo:
  - scheduledTime: {'utc': '2025-11-07 07:00Z', 'local': '2025-11-07 08:00+01:00'}
  - Tipo: <class 'dict'>
DataFrame creado con 815 filas
Primeros valores de scheduled_arrival_time:
0    2025-11-07 07:00Z
1    2025-11-07 07:00Z
2    2025-11-07 07:00Z
3    2025-11-07 07:00Z
4    2025-11-07 07:00Z
Name: scheduled_arrival_time, dtype: object
Tipos: [<class 'str'>]
815/815 fechas convertidas exitosamente
Datos finales:
     scheduled_arrival_time flight_number from_airport  airline     aircraft  \
0 2025-11-07 07:00:00+00:00       VY 5006       Madrid  Vueling  Airbus A321   
1 2025-11-07 07:00:00+00:00       QR 8002       Madrid    Qatar  Airbus A321   
2 2025-11-07 07:00:00+00:00       IB 401S       Madrid   Iberia  Airbus A321   
3 2025-11-07 07:00:00+00:00       AV 6029       Madrid  Avianca  Airbus A321   
4 2025-11-07 07:00:00+00:00       DT 6002       Madrid     TAAG  Airbus A321   

  icao_code  
0      LEBL  
1      LEBL 

In [16]:
# Insertar datos en la base de datos MySQL si hay datos disponibles
if not flights_df.empty:
    # Configuración de conexión a la base de datos
    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = "EvanManx8"  # Asegúrate de que esta contraseña es correcta
    port = 3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    
    try:
        # Enviar datos a la base de datos
        flights_df.to_sql('flight_arrival', 
                         if_exists='append', 
                         con=con, 
                         index=False)
        print("Datos de vuelos insertados correctamente en la base de datos")
        
    except Exception as e:
        print(f"Error al insertar en la base de datos: {e}")
else:
    print("No hay datos para insertar en la base de datos")


Datos de vuelos insertados correctamente en la base de datos
